# Sesión 10

## Introducción a redes bayesianas

> **Objetivos:**
> - Aprender qué es una red Bayesiana y cómo se factorizan distribuciones sobre ellas.

### _¿Qué son los modelos gráficos probabilísticos?_

Los MGPs son un _marco general_ para representar y razonar sobre incertidumbre en sistemas complejos.

El nombre tiene tres palabras clave: **modelos**, **gráficos** y **probabilísticos**.

#### Modelos

* Representación declarativa de cómo entendemos el mundo.
* Permite separar:
    * La construcción del modelo (por humano o mediante aprendizaje automático).
    * De los _algoritmos de inferencia_ (que responden preguntas sobre el modelo).
    * De los _métodos de aprendizaje_ (que ajustan sus parámetros con datos).

> Un "modelo" gráfico probabilístico describe cómo creemos que las cosas se relacionan, sin importar todavía cómo las calculamos.

#### Probabilísticos

Se llama así porque los modelos tratan con **incertidumbre**.

> "Probabilístico" significa que el modelo admite la duda y razona con ella de forma lógica

#### Gráficos

Es `gráfico` porque usamos grafos (nodos y aristas) para representar las dependencias entre variables.

* Cada **nodo** representa una **variable aleatoria**.
* Cada **arista** indica una **relación probabilística (una dependencia directa)**.

Estos grafos permiten:

* Representar distribuciones gigantes de forma compacta (sin enumerar todas las combinaciones posibles).
* Razonar, usando la estructura del grafo para decidir qué variables influyen en cuáles.
* Aprender sus parámetros con pocos datos o cinluso con ayuda de expertos humanos.

> "Gráfico" significa que el modelo usa conexiones visuales (nodos y aristas) para representar dependencias probabilísticas entre variables.

## 1. Preliminares

### 1.1. Ejemplo: Modelo de estudiante

Consideramos el caso de un **estudiante dentro de un curso**.

Queremos razonar acerca de las siguientes variables aleatorias:

- **I** → Inteligencia del estudiante  
- **D** → Dificultad del curso  
- **C** → Calificación del estudiante dentro del curso  
- **P** → Puntaje en las pruebas estatales  
- **R** → Carta de recomendación laboral emitida por el profesor


#### Discretización de variables

**``Inteligencia (I)``**

$$
\mathrm{Val}(I) = \{ i^0, i^1 \}
$$  

  - $i^0$: inteligencia **baja**  
  - $i^1$: inteligencia **alta**


**``Dificultad (D)``**  

$$
\mathrm{Val}(D) = \{ d^0, d^1 \}
$$  

  - $d^0$: curso **fácil**  
  - $d^1$: curso **difícil**


**``Calificación (C)``** 

$$
\mathrm{Val}(C) = \{ c^0, c^1, c^2 \}
$$  

  - $c^0$: **baja** ($C < 6$)  
  - $c^1$: **media** ($6 \leq C < 9$)  
  - $c^2$: **alta** ($C \geq 9$)

**``Puntaje de examen (E)``**

$$
\mathrm{Val}(E) = \{ e^0, e^1 \}
$$  

  - $e^0$: **mal puntaje**  
  - $e^1$: **buen puntaje**

**``Carta de recomendación (R)``** 

$$
\mathrm{Val}(R) = \{ r^0, r^1 \}
$$  

  - $r^0$: carta **débil**  
  - $r^1$: carta **fuerte**

#### ❓ Pregunta

De no usar **modelos gráficos probabilísticos**, _¿cuántos parámetros necesitaríamos para especificar por completo la distribución sobre las cinco variables mencionadas($I, D, C, E, R$)?_

<details>
<summary>Respuesta</summary>

Cada variable puede tomar un cierto número de valores:

$$
|\mathrm{Val}(I)| = 2, \quad
|\mathrm{Val}(D)| = 2, \quad
|\mathrm{Val}(C)| = 3, \quad
|\mathrm{Val}(P)| = 2, \quad
|\mathrm{Val}(R)| = 2
$$

Por tanto, el número total de combinaciones posibles es:

$$
2 \times 2 \times 3 \times 2 \times 2 = 48
$$
 
> Son parámetros **independientes** aquellos cuyo valor **no está completamente determinado** por el valor de otros parámetros.

En este caso, la distribución $P(I,D,C,E,R)$ se especifica con **48 parámetros**, sin embargo, si hablamos de parámetros independientes:

$$
48 - 1 = 47
$$

Si no utilizáramos modelos gráficos probabilísticos, necesitaríamos **47 parámetros independientes** para especificar completamente la distribución conjunta sobre las variables $I, D, C, |, R$.
</details>

#### Red Bayesiana 

Proponemos la siguiente estructura:
- La **inteligencia** $I$ y la **dificultad** $D$ causan la **calificación** $C$.
- La **inteligencia** $I$ influye en el **puntaje** $E$ de pruebas estandarizadas.
- La **calificación** $C$ influye en la **recomendación** $R$.


```{figure} ../images/sesion9-student-model.png
:alt: student-model
:fig-align: center
:width: 700px
```

Al final de esta clase, entenderemos **cómo codificar la distribución de las variables** sobre esta red bayesiana y, a partir de ello, **por qué este modelo necesita muchos menos parámetros** que el modelo genérico (la **distribución conjunta completa**).

### 1.2. Distribuciones de probabilidad y sus operaciones

Consideremos el ejemplo del estudiante, esta vez reducido a las variables $I$, $D$ y $C$.

Una distribución conjunta, $P(I,D,C)$, sobre estas tres variables es:

|  $I$  |  $D$  |  $C$  |  $P$  |
| ----- | ----- | ----- | ----- |
| $i^0$ | $d^0$ | $c^0$ | 0.126 |
| $i^0$ | $d^0$ | $c^1$ | 0.168 |
| $i^0$ | $d^0$ | $c^2$ | 0.126 |
| $i^0$ | $d^1$ | $c^0$ | 0.126 |
| $i^0$ | $d^1$ | $c^1$ | 0.045 |
| $i^0$ | $d^1$ | $c^2$ | 0.009 |
| $i^1$ | $d^0$ | $c^0$ | 0.0056|
| $i^1$ | $d^0$ | $c^1$ | 0.0224|
| $i^1$ | $d^0$ | $c^2$ | 0.252 |
| $i^1$ | $d^1$ | $c^0$ | 0.024 |
| $i^1$ | $d^1$ | $c^1$ | 0.036 |
| $i^1$ | $d^1$ | $c^2$ | 0.06  |

**Pregunta.** ¿Cuántos parámetros en total?


<details>
<summary>Respuesta</summary>

$$2 \times 2 \times 3 = 12$$

</details>

In [1]:
from pgmpy.factors.discrete import JointProbabilityDistribution
from pgmpy.factors.discrete import DiscreteFactor

In [2]:
JointProbabilityDistribution?

Init signature: JointProbabilityDistribution(variables, cardinality, values)
Docstring:      Base class for Joint Probability Distribution
Init docstring:
Initialize a Joint Probability Distribution class.

Defined above, we have the following mapping from variable
assignments to the index of the row vector in the value field:

+-----+-----+-----+-------------------------+
|  x1 |  x2 |  x3 |    P(x1, x2, x2)        |
+-----+-----+-----+-------------------------+
| x1_0| x2_0| x3_0|    P(x1_0, x2_0, x3_0)  |
+-----+-----+-----+-------------------------+
| x1_1| x2_0| x3_0|    P(x1_1, x2_0, x3_0)  |
+-----+-----+-----+-------------------------+
| x1_0| x2_1| x3_0|    P(x1_0, x2_1, x3_0)  |
+-----+-----+-----+-------------------------+
| x1_1| x2_1| x3_0|    P(x1_1, x2_1, x3_0)  |
+-----+-----+-----+-------------------------+
| x1_0| x2_0| x3_1|    P(x1_0, x2_0, x3_1)  |
+-----+-----+-----+-------------------------+
| x1_1| x2_0| x3_1|    P(x1_1, x2_0, x3_1)  |
+-----+-----+-----+-------

In [3]:
p_IDC = JointProbabilityDistribution(
    variables=['I', 'D', 'C'],
    cardinality=[2, 2, 3],
    values=[0.126, 0.168, 0.126, 0.126, 0.045, 0.009, 0.0056, 0.0224, 0.252, 0.024, 0.036, 0.06]
)

In [4]:
print(p_IDC)

+------+------+------+------------+
| I    | D    | C    |   P(I,D,C) |
+======+======+======+============+
| I(0) | D(0) | C(0) |     0.1260 |
+------+------+------+------------+
| I(0) | D(0) | C(1) |     0.1680 |
+------+------+------+------------+
| I(0) | D(0) | C(2) |     0.1260 |
+------+------+------+------------+
| I(0) | D(1) | C(0) |     0.1260 |
+------+------+------+------------+
| I(0) | D(1) | C(1) |     0.0450 |
+------+------+------+------------+
| I(0) | D(1) | C(2) |     0.0090 |
+------+------+------+------------+
| I(1) | D(0) | C(0) |     0.0056 |
+------+------+------+------------+
| I(1) | D(0) | C(1) |     0.0224 |
+------+------+------+------------+
| I(1) | D(0) | C(2) |     0.2520 |
+------+------+------+------------+
| I(1) | D(1) | C(0) |     0.0240 |
+------+------+------+------------+
| I(1) | D(1) | C(1) |     0.0360 |
+------+------+------+------------+
| I(1) | D(1) | C(2) |     0.0600 |
+------+------+------+------------+


In [5]:
# Verificar que la distribución es válida
p_IDC.values.sum()

np.float64(1.0)

In [6]:
isinstance(p_IDC, JointProbabilityDistribution), isinstance(p_IDC, DiscreteFactor)

(True, True)

| Clase | Qué representa | Ejemplo |
| :---- | :------------- | :------ |
| `DiscreteFactor` | Función genérica sobre variables discretas (no necesariamente normalizada) | $\phi(X, Y, Z)$ |
| `JointProbabilityDistribution` | Caso especial de `DiscreteFactor` donde la función es una **probabilidad válida** (suma = 1) | $P(X, Y, Z)$ |

> Toda `JointProbabilityDistribution` **es** un `DiscreteFactor`, pero no todo `DiscreteFactor` **es** una `JointProbabilityDistribution`.

#### ¿Qué operaciones podemos llevar a cabo sobre una distribucción?

**1. Reducción**

Supongamos que observamos que la calificación final del estudiante es alta, esto es, $C=c^2$. La operación de *reducción* consiste en eliminar todas las filas que no son consistentes con la observación:

In [7]:
#p_IDC.reduce?

In [8]:
# Operación de reducción C=c2
p_IDC_reduce_c2 = p_IDC.reduce(values=[('C', 2)], inplace=False)

In [9]:
print(p_IDC_reduce_c2)

+------+------+----------+
| I    | D    |   P(I,D) |
+======+======+==========+
| I(0) | D(0) |   0.1260 |
+------+------+----------+
| I(0) | D(1) |   0.0090 |
+------+------+----------+
| I(1) | D(0) |   0.2520 |
+------+------+----------+
| I(1) | D(1) |   0.0600 |
+------+------+----------+


In [10]:
# Verificar si es una distribución válida
p_IDC_reduce_c2.values.sum()

np.float64(0.447)

Matemáticamente, esta operación equivale a considerar la distribución evaluada

$$P(I, D, C=c^2) = P(I, D, c^2)$$

**Pregunta**. ¿Es este resultado una distribución de probabilidad sobre las variables $I,D$?

**2. Condición**

A partir de la operación de **reducción**, si queremos obtener una distribución legítima sobre las variables que no reducimos, debemos dividir sobre la suma:

In [11]:
#p_IDC.conditional_distribution?

In [12]:
# Operación de condición sobre C=c2
p_IDC_cond_c2 = p_IDC.conditional_distribution([('C', 2)], inplace=False)

In [13]:
print(p_IDC_cond_c2)

+------+------+----------+
| I    | D    |   P(I,D) |
+======+======+==========+
| I(0) | D(0) |   0.2819 |
+------+------+----------+
| I(0) | D(1) |   0.0201 |
+------+------+----------+
| I(1) | D(0) |   0.5638 |
+------+------+----------+
| I(1) | D(1) |   0.1342 |
+------+------+----------+


In [14]:
# Verificar si es una distribución válida
p_IDC_cond_c2.values.sum()

np.float64(1.0)

Matemáticamente, esta operación equivale a considerar la distribución condicionada $P(I, D| C=c^2) = P(I, D| c^2)$.

**Pregunta**. ¿Es este resultado una distribución de probabilidad sobre las variables $I,D$?

**3. Marginalización**

Cuando tenemos una distribución de probabilidad sobre un conjunto de variables y producimos una sobre un subconjunto de las variables originales. Por ejemplo, queremos la distribución marginal sobre $I, D$:

In [15]:
# Imprimir distribución inicial
print(p_IDC)

+------+------+------+------------+
| I    | D    | C    |   P(I,D,C) |
+======+======+======+============+
| I(0) | D(0) | C(0) |     0.1260 |
+------+------+------+------------+
| I(0) | D(0) | C(1) |     0.1680 |
+------+------+------+------------+
| I(0) | D(0) | C(2) |     0.1260 |
+------+------+------+------------+
| I(0) | D(1) | C(0) |     0.1260 |
+------+------+------+------------+
| I(0) | D(1) | C(1) |     0.0450 |
+------+------+------+------------+
| I(0) | D(1) | C(2) |     0.0090 |
+------+------+------+------------+
| I(1) | D(0) | C(0) |     0.0056 |
+------+------+------+------------+
| I(1) | D(0) | C(1) |     0.0224 |
+------+------+------+------------+
| I(1) | D(0) | C(2) |     0.2520 |
+------+------+------+------------+
| I(1) | D(1) | C(0) |     0.0240 |
+------+------+------+------------+
| I(1) | D(1) | C(1) |     0.0360 |
+------+------+------+------------+
| I(1) | D(1) | C(2) |     0.0600 |
+------+------+------+------------+


In [16]:
# Marginalizar I, D
p_IDC_marg_ID = p_IDC.marginalize(variables=['I', 'D'], inplace=False)

In [17]:
print(p_IDC_marg_ID)

+------+--------+
| C    |   P(C) |
+======+========+
| C(0) | 0.2816 |
+------+--------+
| C(1) | 0.2714 |
+------+--------+
| C(2) | 0.4470 |
+------+--------+


Y si queremos la marginal sobre $I$ nada más:

In [18]:
# Marginalizar C y D
p_IDC_marg_CD = p_IDC.marginalize(variables=['C', 'D'], inplace=False)

In [19]:
print(p_IDC_marg_CD)

+------+--------+
| I    |   P(I) |
+======+========+
| I(0) | 0.6000 |
+------+--------+
| I(1) | 0.4000 |
+------+--------+


In [20]:
# Marginalizar C y I
p_IDC_marg_CI = p_IDC.marginalize(variables=['C', 'I'], inplace=False)

In [21]:
print(p_IDC_marg_CI)

+------+--------+
| D    |   P(D) |
+======+========+
| D(0) | 0.7000 |
+------+--------+
| D(1) | 0.3000 |
+------+--------+


Matemáticamente, las anteriores operaciones equivalen a:

$$P(I, D) = \sum_{c\in\mathrm{Val}(C)} P(I, D, C=c), \text{ y }$$

$$P(I) = \sum_{(c,d)\in\mathrm{Val}(C,D)} P(I, D=d, C=c).$$

En abuso de la notación, para no hacer engorrosa la escritura, las anteriores sumas se expresan comúnmente como:

$$P(I, D) = \sum_{C} P(I, D, C), \text{ y }$$

$$P(I) = \sum_{C,D} P(I, D, C).$$

## 2. Fundamentos de redes bayesianas

### 2.1. Modelando independencias

_**¿Qué necesitamos hacer para para que estos nodos y aristas representen una distribución de probabilidad?**_

```{figure} ../images/sesion9-student-model.png
:alt: student-model
:fig-align: center
:width: 700px
```

* **1. Cada nodo tiene su mini tabla de comportamiento (CPD)**

Una CPD _(Conditional Probability Distribution)_ describe cómo se comporta una variable **dado** el estado de sus padres en el grafo.

Por ejemplo:

* $P(D)$ → solo depende de sí misma (no tiene padres).

* $P(I)$ → igual, es independiente.

* $P(C \mid I,D)$ → depende de la inteligencia y la dificultad.

* $P(E \mid I)$ → depende de la inteligencia.

* $P(R \mid C)$ → depende de la calificación.

* **2. Las CPDs son los ladrillos básicos de la red**

Cada nodo de la red tiene su propia CPD -es decir, su propio "bloque de conocimiento local"-.

Si la red tiene $5$ nodos, entonces hay $5$ CPDs.

Al unirlas, no tienes una lista infinita de casos posibles, sino **pequeñas piezas de información localmente coherentes**.

* **3.¿Cómo se unen todas?**

Para obtener la **distribución conjunta completa**, multiplicamos todas las CPDs entre sí.

Esto se llama la _regla de la cadena de las redes bayesianas_.

$$
P(I,D,C,E,R) = P(I) \times P(D) \times P(C \mid I,D) \times P(E \mid I) \times P(R \mid C)
$$

💬 Intuitivamente:

>Empieza con las causas base (I, D) y ve multiplicando los efectos condicionales según el grafo.

* **4. ¿qué estamos haciendo realmente?**

Matemáticamente, lo que hacemos al multiplicar las CPDs es un **producto de factores.**

Cada CPD es un **factor** (una tabla con valores numéricos).

Las CPDs *comparten variables* (por ejemplo, $C$ aparece en dos CPDs: $P(C \mid I,D)$ y $P(R \mid C)$), así que al multiplicarlas obtenemos un **factor grande**, cuya **alcance** _scope_ incluye **todas las variables** de la red.

* **5. ¿Por qué esto es tan poderoso?**

Porque...
- No necesitas escribir $48$ números distintos.
- Solo defines unas pocas tablas locales.
- La estructura del grafo ya codifica las **independencias** entre variables.

Así, puedes reconstruir la distribución completa de una manera compacta y comprensible.

| Elemento | Significado | Ejemplo |
|-----------|--------------|----------|
| **Nodo sin padres** | Distribución simple | $P(I)$, $P(D)$ |
| **Nodo con padres** | CPD (condicional) | $P(C \mid I, D)$ |
| **Toda la red** | Producto de todas las CPDs | $P(I)P(D)P(C \mid I,D)P(E \mid I)P(R \mid C)$ |
| **Resultado final** | Distribución conjunta completa | $P(I,D,C,E,R)$ |


```{figure} ../images/sesion9-student-model-factors.png
:alt: student-model-factors
:fig-align: center
:width: 800px
```

<details>
<summary>💡 Justificación formal: regla de la cadena + independencia</summary>

$$
P(I, D, C, E, R) = P(I) \times P(D) \times P(C \mid I, D) \times P(E \mid I) \times P(R \mid C)
$$

Este producto no es casual: surge directamente de la **regla de la cadena de la probabilidad**, combinada con las **independencias condicionales** que nos indica la estructura del grafo.


<details>
<summary>🔹 1. Aplicamos la regla de la cadena</summary>


La **regla de la cadena** nos dice que cualquier distribución conjunta se puede descomponer como el producto de probabilidades condicionales:

$$
P(D, I, C, E, R) = P(D) \, P(I \mid D) \, P(C \mid D, I) \, P(E \mid D, I, C) \, P(R \mid D, I, C, E)
$$

Esto es **una identidad matemática**, sin asumir ninguna independencia aún.

</details>

<details>
<summary>🔹 2. Aplicamos las independencias locales del grafo</summary>

De la estructura del grafo, sabemos que:

- $D$ y $I$ **no tienen padres** → $P(D)$ y $P(I)$  
- $C$ depende de $D$ e $I$ → $P(C \mid D, I)$  
- $E$ depende solo de $I$ → $P(E \mid I)$  
- $R$ depende solo de $C$ → $P(R \mid C)$  

Además, $D$ e $I$ son **independientes entre sí**.

</details>

<details>
<summary>🔹 3. Sustituimos las dependencias en la regla de la cadena</summary>

Comenzamos con la regla general:

$$
P(D, I, C, E, R) = P(D) \, P(I \mid D) \, P(C \mid D, I) \, P(E \mid D, I, C) \, P(R \mid D, I, C, E)
$$

Ahora, aplicamos las **independencias** del grafo:

1. $P(I \mid D) = P(I)$  (porque $I$ y $D$ son independientes)  
2. $P(E \mid D, I, C) = P(E \mid I)$  (porque $E$ solo depende de $I$)  
3. $P(R \mid D, I, C, E) = P(R \mid C)$  (porque $R$ solo depende de $C$)

</details>

<details>
<summary>🔹 4. Factorización final del modelo</summary>

Sustituyendo esas simplificaciones, obtenemos:

$$
P(D, I, C, E, R) = P(D) \, P(I) \, P(C \mid D, I) \, P(E \mid I) \, P(R \mid C)
$$

</details>

</details>

```{admonition} 🔹 Definición forma de una red bayesiana
:class: tip

Hasta ahora hemos visto las redes bayesianas de forma intuitiva: como **grafos donde cada nodo representa una variable** y **cada flecha una relación de dependencia**. 

Ahora podemos formalizar esta idea.

**Definición**

Una **red bayesiana** es un **grafo dirigido acíclico (DAG)** $\mathcal{G}$, donde:

- Cada nodo representa una **variable aleatoria** $X_i$.
- Cada nodo se asocia con una **distribución condicional**  
  $P(X_i \mid Pa_\mathcal{G}(X_i))$, donde $Pa_\mathcal{G}(X_i)$ son los **padres** de $X_i$ en el grafo.
- Cada arco indica una **influencia causal o probabilística directa**.

La **distribución conjunta** sobre todas las variables se obtiene multiplicando todas las distribuciones locales:

$$
P(X_1, X_2, \dots, X_n) = \prod_{i=1}^{n} P(X_i \mid Pa_\mathcal{G}(X_i))
$$
```

#### De la factorización a las independencias locales

Acabamos de ver que la distribución conjunta puede escribirse como un **producto de probabilidades condicionales**:

$$
P(I, D, C, E, R) = P(I) \, P(D) \, P(C \mid I, D) \, P(E \mid I) \, P(R \mid C)
$$

El grafo no solo nos dice *quién depende de quién*, sino también *quién **no** depende de quién* una vez que conocemos ciertas variables.

#### Independencias locales en una red bayesiana

Cada nodo de una red bayesiana “vive” en su propio contexto local: solo depende de sus **padres** y es **independiente del resto del grafo** si ya conocemos el valor de esos padres.

Formalmente, para cada variable $X_i$:

$$
X_i \perp NoDescendientes_\mathcal{G}(X_i) \mid Pa_\mathcal{G}(X_i)
$$

Esto se conoce como la **independencia local** de $X_i$.

> Si sabemos las causas directas de algo, el resto del sistema no cambia su comportamiento.

Por ejemplo, si $C$ tiene como padres a $D$ e $I$, entonces $C$ es independiente de otros nodos como $E$, una vez que conocemos $D$ e $I$.

#### Actividad

1. Obtener las independencias locales que codifica la red Bayesiana del estudiante.

2. Comparar el número de parámetros independientes que necesita la red Bayesiana del estudiante contra el número de parámetros que necesitaría la distribución conjunta sin ninguna suposición de independencia.

**¿Cómo declarar una red Bayesiana en pgmpy?**

In [22]:
from pgmpy.models import BayesianNetwork, DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD

In [23]:
#DiscreteBayesianNetwork?

#### 1. Definimos los arcos del grafo

In [24]:
student_model = DiscreteBayesianNetwork(
    [("D", "C"), ("I", "C"), ("I", "E"), ("C", "R")]
)

#### 2. Definimos las CPDs de cada nodo

In [25]:
# Definimos distribución condicional de D
cpd_D = TabularCPD(
    variable='D',
    variable_card=2,
    values=[
        [0.6],
        [0.4]
    ]
)
# Definimos distribución condicional de I
cpd_I = TabularCPD(
    variable='I',
    variable_card=2,
    values=[
        [0.7],
        [0.3]
    ]
)

In [26]:
print(cpd_D)

+------+-----+
| D(0) | 0.6 |
+------+-----+
| D(1) | 0.4 |
+------+-----+


La representación de las distribuciones condicionales en `pgmpy` es un poquito distinto a como está en la tabla de arriba. En `pgmpy` las columnas representan evidencia y las filas los distintos estados de la variable en la distribución condicional:

|         | $i^0 d^0$ | $i^0 d^1$ | $i^1 d^0$ | $i^1 d^1$ |
| ------- | --------- | --------- | --------- | --------- |
| $c^0$   | 0.3       | 0.7       | 0.02      | 0.2       |
| $c^1$   | 0.4       | 0.25      | 0.08      | 0.3       |
| $c^2$   | 0.3       | 0.05      | 0.9       | 0.5       |

In [ ]:
# Definimos distribución condicional de C
cpd_C = TabularCPD(
    variable='C',
    variable_card=3,
    values=[
        [0.30, 0.70, 0.02, 0.20],
        [0.40, 0.25, 0.08, 0.30],
        [0.30, 0.05, 0.90, 0.50]
    ],
    evidence=['I', 'D'], #<--- ojo aquí: los padres de C son I y D
    evidence_card=[2, 2] #<--- ojo aquí: las cardinalidades de I y D son 2 y 2 respectivamente
)
# Definimos distribución condicional de E
cpd_E = TabularCPD(
    variable='E',
    variable_card=2,
    values=[
        [0.95, 0.20],
        [0.05, 0.80]
    ],
    evidence=['I'],
    evidence_card=[2]
)
# Definimos distribución condicional de R
cpd_R = TabularCPD(
    variable='R',
    variable_card=2,
    values=[
        [0.99, 0.40, 0.10],
        [0.01, 0.60, 0.90]
    ],
    evidence=['C'],
    evidence_card=[3]
)

In [28]:
print(cpd_R)

+------+------+------+------+
| C    | C(0) | C(1) | C(2) |
+------+------+------+------+
| R(0) | 0.99 | 0.4  | 0.1  |
+------+------+------+------+
| R(1) | 0.01 | 0.6  | 0.9  |
+------+------+------+------+


#### 3. Añadimos las CPDs a la red y verificamos el modelo

In [29]:
#student_model.add_cpds?

In [30]:
# Asociamos las distribuciones condicionales a la red
student_model.add_cpds(cpd_D, cpd_I, cpd_C, cpd_E, cpd_R)

#### 4. Verificar que el modelo es válido

In [31]:
help(student_model.check_model)

Help on method check_model in module pgmpy.models.DiscreteBayesianNetwork:

check_model() method of pgmpy.models.DiscreteBayesianNetwork.DiscreteBayesianNetwork instance
    Check the model for various errors. This method checks for the following
    errors.

    * Checks if the sum of the probabilities for each state is equal to 1 (tol=0.01).
    * Checks if the CPDs associated with nodes are consistent with their parents.

    Returns
    -------
    check: boolean
        True if all the checks pass otherwise should throw an error.



In [32]:
student_model.check_model()

True

### Independencias locales

Una vez tenemos el modelo, podemos hacer varias cosas con él. Entre ellas, podemos verificar las independencias locales que codifica el modelo:

In [33]:
#student_model.local_independencies?

> Cada nodo es independiente de sus no descendientes, dados sus padres.

| Nodo | Padres (condición) | Descendientes | No descendientes | Independencia local |
|:------:|:------------------:|:------------------:|:----------------------:|:---------------------------:|
| **D (Dificultad)** | — | C, R | I, E | $D \perp \{I, E\}$ |
| **I (Inteligencia)** | — | C, R, E | D | $I \perp D$ |
| **C (Calificación)** | D, I | R | E | $C \perp E \mid D, I$ |
| **E (Prueba)** | I | — | D, C, R | $E \perp \{D, C, R\} \mid I$ |
| **R (Carta)** | C | — | D, I, E | $R \perp \{D, I, E\} \mid C$ |


In [34]:
# La variable D es independiente de I  y de E.
student_model.local_independencies('D')

(D ⟂ E, I)

In [35]:
student_model.local_independencies('C')

(C ⟂ E | D, I)

In [36]:
student_model.local_independencies(["D", "I", "C", "E", "R"])

(D ⟂ E, I)
(I ⟂ D)
(C ⟂ E | D, I)
(E ⟂ R, C, D | I)
(R ⟂ E, D, I | C)

## 2.2. Patrones de razonamiento e inferencia

Teniendo una situación modelada con una red Bayesiana, nos podemos plantear **tres** tipos básicos de razonamiento de podríamos querer resolver:

* Razonamiento causal
* Razonamiento evidencial
* Razonamiento intercausal

**1. Razonamiento causal**

El **razonamiento causal** sigue la dirección natural de las flechas del grafo: va de **causa → efecto**, o de **nodo padre → nodo hijo**.

> Si sé algo sobre las causas, ¿qué puedo inferir sobre sus efectos?

![causal-reasoning](../images/sesion10-student-model-causal.png)

Por ejemplo, 

**Pregunta**: ¿cuál es la probabilidad de obtener una buena carta de recomendación?

$$P(r^1) = \sum_{D,I,C,E} P(D,I,C,E,r^1) \approx ?$$

In [37]:
# Obtenemos la distribución conjunta de la red
p_joint = (
    cpd_I.to_factor()
    * cpd_D.to_factor()
    * cpd_C.to_factor()
    * cpd_E.to_factor()
    * cpd_R.to_factor()
)

In [38]:
print(p_joint)

+------+------+------+------+------+------------------+
| R    | I    | E    | C    | D    |   phi(R,I,E,C,D) |
+======+======+======+======+======+==================+
| R(0) | I(0) | E(0) | C(0) | D(0) |           0.1185 |
+------+------+------+------+------+------------------+
| R(0) | I(0) | E(0) | C(0) | D(1) |           0.1843 |
+------+------+------+------+------+------------------+
| R(0) | I(0) | E(0) | C(1) | D(0) |           0.0638 |
+------+------+------+------+------+------------------+
| R(0) | I(0) | E(0) | C(1) | D(1) |           0.0266 |
+------+------+------+------+------+------------------+
| R(0) | I(0) | E(0) | C(2) | D(0) |           0.0120 |
+------+------+------+------+------+------------------+
| R(0) | I(0) | E(0) | C(2) | D(1) |           0.0013 |
+------+------+------+------+------+------------------+
| R(0) | I(0) | E(1) | C(0) | D(0) |           0.0062 |
+------+------+------+------+------+------------------+
| R(0) | I(0) | E(1) | C(0) | D(1) |           0

In [39]:
# Marginalizar sobre las variables I, D, C, E
p_R = p_joint.marginalize(variables=['I', 'D', 'C', 'E'], inplace=False)
print(p_R)

+------+----------+
| R    |   phi(R) |
+======+==========+
| R(0) |   0.4977 |
+------+----------+
| R(1) |   0.5023 |
+------+----------+


In [40]:
p_r1 = p_R.reduce(values=[('R', 1)], inplace=False)
print(p_r1)

+---------+
|   phi() |
+=========+
|  0.5023 |
+---------+


Sin embargo, podemos evaluar cómo esta probabilidad cambia si la condicionamos sobre la inteligencia. Por ejemplo, si el estudiante no es muy inteligente

$$P(r^1 | i^0) = \frac{P(r^1, i^0)}{P(i^0)} = \frac{\sum_{D,C,E} P(D, i^0, C, E, r^1)}{\sum_{D,C,E,R} P(D, i^0, C, E, R)} \approx ?$$

In [41]:
p_RI = p_joint.marginalize(variables=['D', 'C', 'E'], inplace=False)
print(p_RI)

+------+------+------------+
| R    | I    |   phi(R,I) |
+======+======+============+
| R(0) | I(0) |     0.4280 |
+------+------+------------+
| R(0) | I(1) |     0.0697 |
+------+------+------------+
| R(1) | I(0) |     0.2720 |
+------+------+------------+
| R(1) | I(1) |     0.2303 |
+------+------+------------+


In [42]:
i_0 = p_RI.get_value(R=1, I=0)
i_0

np.float64(0.27201999999999993)

In [43]:
p_I = p_joint.marginalize(variables=['D', 'C', 'E', 'R'], inplace=False)
print(p_I)

+------+----------+
| I    |   phi(I) |
+======+==========+
| I(0) |   0.7000 |
+------+----------+
| I(1) |   0.3000 |
+------+----------+


In [44]:
r1_i0 = p_I.get_value(I=0)
r1_i0

np.float64(0.7)

**¿se esperaba esto o no?**

In [45]:
print('probabilidad de r1 dado i0:', i_0 / r1_i0)

probabilidad de r1 dado i0: 0.38859999999999995


Por otra parte, si también condicionamos sobre la dificultad

$$P(r^1 | i^0, d^0) = \frac{P(r^1, i^0, d^0)}{P(i^0, d^0)} = \frac{\sum_{C,P} P(d^0, i^0, C, E, r^1)}{\sum_{C,P,R} P(d^0, i^0, C, E, R)} \approx ?$$

In [46]:
p_RID = p_joint.marginalize(variables=['C', 'E'], inplace=False)
r1_i0_d0 = p_RID.get_value(R=1, I=0, D=0)
r1_i0_d0

np.float64(0.21546)

In [47]:
p_ID = p_joint.marginalize(variables=['C', 'E', 'R'], inplace=False)
i0_d0 = p_ID.get_value(I=0, D=0)
i0_d0

np.float64(0.41999999999999993)

**¿Se esperaba esto o no?**

In [48]:
print('probabilidad de r1 dado i0 y d0:', r1_i0_d0 / i0_d0)

probabilidad de r1 dado i0 y d0: 0.5130000000000001


---

**2. Razonamiento evidencial**

Va **de efecto a causa**, en sentido contrario a las flechas.

> Si observo un efecto, ¿qué puedo inferir sobre sus causas?

![causal-reasoning](../images/sesion10-student-model-evid.png)

Por ejemplo, la probabilidad de que el curso sea difícil es:

$$P(d^1) = 0.4$$

Condicionando sobre la calificación:

$$P(d^1 | c^0) = \frac{P(d^1, c^0)}{P(c^0)} = \frac{\sum_{I,E,R} P(d^1, I, c^0, E, R)}{\sum_{D,I,E,R} P(D, I, c^0, E, R)} \approx?$$

In [49]:
p_DC = p_joint.marginalize(
    variables=['I', 'E', 'R'],
    inplace=False)
p_i1_c0 = p_DC.get_value(D=1, C=0)
p_i1_c0

np.float64(0.22)

In [50]:
p_C = p_joint.marginalize(
    variables=['D', 'I', 'E', 'R'],
    inplace=False
)
p_c0 = p_C.get_value(C=0)
p_c0

np.float64(0.34959999999999997)

In [51]:
pi1_c0 = p_i1_c0 / p_c0
pi1_c0

np.float64(0.6292906178489703)

In [52]:
#Otra forma de calcular P(D1 | C0) usando inferencia en la red bayesiana    
from pgmpy.inference import VariableElimination
infer = VariableElimination(student_model)
phi = infer.query(
    variables=['D'],
    evidence={'C': 0})
phi.values[1]

np.float64(0.6292906178489702)

> Intuición: observar una calificación baja hace más probable que el curso haya sido difícil (sube de $0.4$ a $\approx 0.63$).

---

Similarmente, la probabilidad de que el estudiante sea inteligente es:

$$P(i^1) = 0.3$$

Condicionando sobre la calificación:

$$P(i^1 | c^0) = \frac{P(i^1, c^0)}{P(c^0)} = \frac{\sum_{D,E,R} P(D, i^1, c^0, E, R)}{\sum_{D,I,E,R} P(D, I, c^0, E, R)} \approx ?$$

In [53]:
p_IC = p_joint.marginalize(
    variables=['D', 'E', 'R'],
    inplace=False)
p_i1_c0 = p_IC.get_value(I=1, C=0)

p_C = p_joint.marginalize(
    variables=['D', 'I', 'E', 'R'],
    inplace=False
)
p_c0 = p_C.get_value(C=0)

In [54]:
p_i1_given_c0 = p_i1_c0 / p_c0
p_i1_given_c0

np.float64(0.07894736842105264)

In [55]:
infer = VariableElimination(student_model)
phi = infer.query(
    variables=['I'],
    evidence={'C': 0})
phi.values[1]

np.float64(0.07894736842105264)

> Intuición: observar una calificación baja hace menos probable que el estudiante sea inteligente (baja del $0.3$ a $\approx 0.11$).

**3. Razonamiento intercausal**

Ocurre cuando **dos causas comparten un mismo efecto** y una de ellas se observa.

> Si conozco una causa, ¿cómo cambia mi creencia sobre la otra, dado que comparten el mismo efecto?


![intercausal-reasoning](../images/sesion10-student-model-inter.png )

$\text{Dificultad} \longrightarrow \text{Calificación} \longleftarrow \text{Inteligencia}$

Normalmente, $D$ e $I$ son independientes. Pero, una vez que conocemos el efecto común -por ejemplo, la calificación $C$-, dejan de serlo.

> Si sabemos que la calificación fue alta y que el curso era difícil, es más probable que el estudiante haya sido inteligente.

Antes de observar $C$:

$$ D \perp I $$

Después de observar $C$:
$$ D \not\perp I \mid C $$

De nuevo, la probabilidad de que el estudiante sea inteligente es:

$$P(i^1) = 0.3$$

Condicionando sobre la calificación:

$$P(i^1 | c^0) = \frac{P(i^1, c^0)}{P(c^0)} \approx 0.07$$

Aún más, si condicionamos sobre la dificultad:

$$P(i^1 | c^0, d^1) = \frac{P(i^1, c^0, d^1)}{P(c^0, d^1)} \approx ?$$

In [56]:
infer = VariableElimination(student_model)
phi = infer.query(
    variables=['I'],
    evidence={'C': 0, 'D': 1})
phi.values[1]

np.float64(0.1090909090909091)

> Inicialmente, el estudiante tiene una probabilidad moderada de ser inteligente ($P(i^1)=0.3$). Al observar que obtuvo una **mala calificación**, esa creencia **disminuye drásticamente** ($P(i^1 \mid c^0) \approx 0.07$). Sin embargo, si además sabemos que el curso era **difícil**, parte de la mala nota se explica por la dificultad, por lo que la probabilidad de que sea inteligente **vuelve a subir ligeramente** $P(i^1 \mid c^0, d^1) \approx 0.11$.

In [ ]:
#guardar el modelo
#import pickle

#with open('student-model.pkl', 'wb') as f:
#    pickle.dump(student_model, f)